In [1]:
import collections
import math
import os
import random
import zipfile

In [2]:
import urllib


In [3]:
import numpy as np
import tensorflow as tf

In [4]:
DOWNLOAD_FILENAME = 'sampleText.zip'

def maybe_download(url_path,expected_bytes):
    if not os.path.exists(DOWNLOAD_FILENAME):
        filename,_ = urllib.request.urlretrieve(url_path,DOWNLOAD_FILENAME)
        
    satatinfo =os.stat(DOWNLOAD_FILENAME)
    if satatinfo.st_size == expected_bytes:
        print('found and verified file :',url_path)
        print('DOWNLOAD_FILE :',DOWNLOAD_FILENAME)
    else:
        print(statinfo.st_size)
        rasie ('Faild to verify file')

In [5]:
def read_words():
    with zipfile.ZipFile(DOWNLOAD_FILENAME) as f:
        firstFile =f.namelist()[0]
        fileString =tf.compat.as_str(f.read(firstFile))
        words = fileString.split()
    return words


In [6]:
url_path ='http://mattmahoney.net/dc/text8.zip'
FILESIZE = 31344016
maybe_download(url_path,FILESIZE)

found and verified file : http://mattmahoney.net/dc/text8.zip
DOWNLOAD_FILE : sampleText.zip


In [7]:
vocabulary = read_words()
print(len(vocabulary))

17005207


In [8]:
def build_dataSet(words,n_words):
    word_counts = [['unknown',-1]]
    counter = collections.Counter(words)
    word_counts.extend(counter.most_common( n_words - 1))
    dictonary = dict()
    for word ,_ in word_counts:
        dictonary[word] = len(dictonary)
        
    word_indxes = list()
    unknown_count = 0
    
    for word in words:
        if word in dictonary:
            index = dictonary[word]
        else:
            index = 0
            unknown_count +=1
        word_indxes.append(index)
    word_counts[0][1] = unknown_count
    
    reversed_dictionary = dict(zip(dictonary.values(),dictonary.keys()))
    return word_counts ,word_indxes ,dictonary ,reversed_dictionary

In [9]:
vocabulary_size = 5000
word_counts ,word_indxes ,dictonary ,reversed_dictionary = build_dataSet(vocabulary,vocabulary_size)



In [10]:
del vocabulary


In [11]:
global_index = 0

In [12]:
def generator_batch(word_indxes,batch_size,num_skips,skip_window):
    global global_index
    
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    
    batch = np.ndarray(shape=(batch_size),dtype = np.int32)
    lables = np.ndarray(shape=(batch_size ,1) ,dtype=np.int32)
    
    span = 2 * skip_window + 1
    
    buffer = collections.deque(maxlen=span)
 
    for _ in range(span):
        buffer.append(word_indxes[global_index])
        global_index = (global_index + 1) % len(word_indxes)
    
    for i in range(batch_size //num_skips):
        target = skip_window
        targets_to_avoid = [skip_window]
        
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0,span - 1)
            
            targets_to_avoid.append(target)
            
            batch[i * num_skips + j] = buffer[skip_window]
            lables[i * num_skips + j, 0] = buffer[target]
            
        buffer.append(word_indxes[global_index])
        global_index = (global_index + 1) % len(word_indxes)
        
    global_index = (global_index + len(word_indxes) -span )% len(word_indxes)
        
    return batch,lables

In [13]:
batch ,labels = generator_batch(word_indxes ,10,2,5)


## ready to set up nural network using tensorflow 

In [14]:
# reset global index becasue we updated while testign the batch code
global_index = 0
valid_size = 16
valid_window = 100
valid_example = np.random.choice(valid_window,valid_size,replace= False)


In [15]:
num_samples = 64
# number of corrupted sample pairs to feed into the NCE

In [16]:
batch_size = 128
embedding_size = 50
skip_window = 2 
num_skips = 2

In [17]:
tf.reset_default_graph()

train_inputs = tf.placeholder(tf.int32 ,shape=[batch_size])
train_labels = tf.placeholder(tf.int32,shape=[batch_size ,1])


In [18]:
validate_dataset = tf.constant(valid_example ,dtype =tf.int32)

In [19]:
embeddings = tf.Variable(
                tf.random_uniform([vocabulary_size,embedding_size], -1.0, 1.0))
embed = tf.nn.embedding_lookup(embeddings,train_inputs)


In [20]:
nce_weights = tf.Variable(
    tf.truncated_normal([vocabulary_size ,embedding_size],stddev=1.0 / math.sqrt(embedding_size))
)
nce_biase = tf.Variable(tf.zeros([vocabulary_size]))

In [21]:
loss =tf.reduce_mean(tf.nn.nce_loss(
    weights = nce_weights,
    biases  = nce_biase,
    labels  = train_labels,
    inputs  = embed,
    num_sampled = num_samples,
    num_classes = vocabulary_size
))

In [22]:
optimizer =tf.train.GradientDescentOptimizer(0.1).minimize(loss)

In [23]:
l2_norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings),1,keep_dims =True))

normalized_embeddings = embeddings /l2_norm

In [24]:
validation_embeddings = tf.nn.embedding_lookup(normalized_embeddings,validate_dataset)

In [25]:
similarity = tf.matmul(validation_embeddings,normalized_embeddings,transpose_b=True)

In [26]:
init = tf.global_variables_initializer()


## noise contrastive estimator to measssure loss


In [27]:
num_steps =20001

In [28]:

with tf.Session() as session:
    init.run()
    
    average_loss = 0
    for step in range(num_steps):
        batch_inputs ,batch_labels = generator_batch(word_indxes, batch_size,num_skips,skip_window)
        
        feed_dict = {train_inputs :batch_inputs, train_labels:batch_labels}
        
        _,loss_val = session.run([optimizer,loss] , feed_dict =feed_dict)
        average_loss += loss_val
        
        if step % 2000 ==0:
            if step >0:
                average_loss /=2000
            
            print('average loss at step ',step," : ",average_loss)
            average_loss = 0
            
        if step % 10000 ==0 :
            sim = similarity.eval()
            
            for i in range(valid_size):
                valid_word = reversed_dictionary[valid_example[i]]
                top_k =8
                
                nearest = (-sim[i,:]).argsort()[1:top_k+1]
                log_str = "nerest to " + valid_word
                
                for k in range(top_k):
                    close_word = reversed_dictionary[nearest[k]]
                    log_str = '%s %s ,' % (log_str,close_word)
                print(log_str)
            print("\n")
    final_embedding = normalized_embeddings.eval()

average loss at step  0  :  200.609985352
nerest to no ban , peer , planet , few , equally , foods , worked , pope ,
nerest to on itself , market , farm , large , artillery , identified , eye , bell ,
nerest to united a , chad , possibility , diseases , chapters , defend , excellent , dedicated ,
nerest to known over , ended , prisoners , mm , indians , advocate , children , publicly ,
nerest to but benjamin , missile , explained , nine , telecommunications , raise , americans , letters ,
nerest to other diet , planet , transferred , napoleon , rival , mainland , giving , tiny ,
nerest to he detroit , originated , extensive , prominent , colleges , employees , franklin , isle ,
nerest to has adults , none , gain , located , pattern , differences , iraq , contemporary ,
nerest to many nonetheless , intel , requirement , inventor , o , formal , markets , siege ,
nerest to seven seat , beam , account , graphic , kosovo , trace , contest , historically ,
nerest to will merely , anderson , 

In [29]:
def save_words_embeddings(embeddings,word_counts):
    np.save('embeddings.npy',embeddings)
    
    words=[x[0] for x in word_counts]
    np.save('words.npy',words)

save_words_embeddings(final_embeddings,data)
    
    